In [1]:
import numpy as np
import pandas as pd

import pickle
import os
from pathlib import Path
import glob
import json
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from PIL import Image
#%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)

sns.set_style('darkgrid')

from scipy.interpolate import interp1d

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory
from xyz10.compute_f_mod import compute_step_positions, compute_step_positions_mod, split_ts_seq
from xyz10.compute_f_mod import compute_steps, compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions
from xyz10.compute_f_mod import correct_positions,  correct_positions_mod

Downsample Rotation, Magnetic, and (to be calculated) Relative Steps (x,y) into WiFi timestamps   

> Setting bssid list

In [2]:
test_bssid = pickle.load(open("./data_out/test_bssid_ranks.pkl", "rb"))
train_bssid = pickle.load(open("./data_out/train_24IDs_standardF_bssid_ranks.pkl", "rb"))

bssid = {}
for site in test_bssid["count"]:
    _test_bssid = test_bssid["count"][site].bssid
    _train_bssid = train_bssid["count"][site].bssid
    _mix_bssid = pd.concat([_test_bssid,  _train_bssid]).unique()
    
    _list = _mix_bssid.tolist()
    
    bssid[site] = _list

> Interpolated STEPS (snap to steps; where steps 0.5s/wifi 0.5-1s dt = expected error is 0.5step=0.25m): Convert/Append/Save each trace per site to DataFrame: Features=BSSID/RSSI + DELAYS + F + M + R + rx + ry + TRACEID + rel time  
> Targets=XY (BSSID not in trace gets min-RSSI and max-DELAY per site) 

In [6]:
wifi_data_train = pickle.load(open("./data_out/train_24IDs_standardF_wifi.pkl", "rb"))

In [7]:
accel_data = pickle.load(open("./data_out/train_24IDs_standardF_motion.pkl", "rb"))

rotate_data = pickle.load(open("./data_out/train_24IDs_standardF_rotate.pkl", "rb"))
magnetic_data = pickle.load(open("./data_out/train_24IDs_standardF_magnetic.pkl", "rb"))

steps_data = pickle.load(open("./data_out/train_24IDs_standardF_steps.pkl", "rb"))

In [7]:
num_s = 0
for site_id in ["5da1389e4db8ce0c98bd0547"]:#tqdm(bssid.keys()): # over sites
    num_s += 1
    print(f"Processing #{num_s}: Site-{site_id} with {len(steps_data[site_id])} traces and {len(bssid[site_id])} bssids")
    site_trace_record_wifi = []

    col_names = []
    for feature_bssid in bssid[site_id]:  # create features-columns list
        col_names.append(feature_bssid)
        col_names.append(feature_bssid+"_D")

    for trace_id in tqdm(steps_data[site_id]):  # over traces

        wifi_record = wifi_data_train[site_id][trace_id][1].copy()
        steps_record = steps_data[site_id][trace_id].copy()

        accel_record = accel_data[site_id][trace_id][1]
        rotate_record = rotate_data[site_id][trace_id][1]
        magnetic_record = magnetic_data[site_id][trace_id][1]       

        trace_record_wifi = np.zeros((steps_record[1].shape[0], len(col_names)))

        for col_id, _ in enumerate(col_names):
            if col_id%2 == 0:
                trace_record_wifi[:, col_id] = int(train_bssid["min_rssi"][site_id])
            else:
                trace_record_wifi[:, col_id] = int(train_bssid["max_delay"][site_id])

        trace_record_wifi = pd.DataFrame(trace_record_wifi, columns=col_names)

        # time, absolute x/y positions of steps, floor number
        trace_record_wifi[["time", "x", "y"]] = steps_record[1][["time", "x", "y"]].copy()
        trace_record_wifi["f"] = steps_record[0].FloorName[0]
        timestamps = trace_record_wifi.time.to_numpy()
        # magnetic field intensity
        magnetic_intensity = np.sqrt(np.square(magnetic_record.x_axis.to_numpy()) + np.square(magnetic_record.y_axis.to_numpy()) + np.square(magnetic_record.z_axis.to_numpy()))
        trace_record_wifi["m"] = interp1d(magnetic_record.time.to_numpy(), magnetic_intensity, kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # z_axis rotation angle (i.e. inplane direction)
        trace_record_wifi["r"] = interp1d(rotate_record.time.to_numpy(), rotate_record.z_axis.to_numpy(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # relative x/y steps positions
        step_timestamps, step_indexs, step_acce_max_mins = compute_steps(accel_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy())
        rel_step_positions = pd.DataFrame(compute_rel_positions(compute_stride_length(step_acce_max_mins),
                                                                compute_step_heading(step_timestamps,
                                                                                     compute_headings(rotate_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy()))), 
                                          columns=["time", "x", "y"])
        # recalculate relative steps for a given timestamps(cumsum->interpolate->differences)
        _rt = np.insert(rel_step_positions.time.to_numpy(), 0, 0.0)
        rx_cum = interp1d(_rt, np.insert(rel_step_positions.x.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        ry_cum = interp1d(_rt, np.insert(rel_step_positions.y.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        trace_record_wifi["rx"] = np.diff(rx_cum, prepend=0.0) 
        trace_record_wifi["ry"] = np.diff(ry_cum, prepend=0.0) 
            
        # trace id
        trace_record_wifi["trace"] = trace_id
        # wifi rssi and delays
        # Take wifi data the nearest to each waypoint
        for j, t0 in enumerate(trace_record_wifi.time):  # over times in trace/for each waypoint

            wifi_record["time0"] = wifi_record.time - t0
            wifi_record["time0"] = wifi_record.time0.apply(abs)

            wifi_min = wifi_record[wifi_record.time0 == wifi_record.time0.min()]
            wifi_min_bssid_list = wifi_min.bssid.tolist()
            wifi_min_delay_list = [col+"_D" for col in wifi_min_bssid_list]

            trace_record_wifi.loc[j, wifi_min_bssid_list] = wifi_min.rssi.to_numpy()
            trace_record_wifi.loc[j, wifi_min_delay_list] = wifi_min.delay.to_numpy().astype(int)    

        site_trace_record_wifi.append(trace_record_wifi)
        #break  # only fist trace

    site_trace_record_wifi = pd.concat(site_trace_record_wifi, ignore_index=True)
        
    # saving model for a given site_id
    with open(f"./data_out/full24/{site_id}_10k_mix-counts.pkl", "wb") as f:
        pickle.dump(site_trace_record_wifi, f)
    #break  # only first site

Processing #1: Site-5da1389e4db8ce0c98bd0547 with 514 traces and 1041 bssids


  0%|          | 0/514 [00:00<?, ?it/s]

Convert Train data into sequences of 5-10-20 records (for LSTM)

In [8]:
site_ids = {"5d27075f03f801723c2e360f": (73141, 14063), "5dc8cea7659e181adb076a3f": (57849, 9733), "5dbc1d84c1eb61796cf7c010": (61727, 9043), "5da138b74db8ce0c98bd4774": (56668, 7075),
            "5d2709c303f801723c3299ee": (33368, 11667), "5da958dd46f8266d0737457b": (47796, 7003), "5d2709bb03f801723c32852c": (44009, 4909), "5d27096c03f801723c31e5e0": (19337, 9933),
            "5a0546857ecc773753327266": (26532, 6799), "5c3c44b80379370013e0fd2b": (29359, 6131), "5d27097f03f801723c320d97": (35121, 4985), "5da1382d4db8ce0c98bbe92e": (28975, 5729),
            "5d2709b303f801723c327472": (32449, 3831), "5d2709d403f801723c32bd39": (23545, 4283), "5da138764db8ce0c98bcaa46": (27771, 3781), "5da1383b4db8ce0c98bc11ab": (34396, 3055),
            "5d2709e003f801723c32d896": (29752, 2623), "5da138754db8ce0c98bca82f": (13795, 3259), "5da1389e4db8ce0c98bd0547": (17795, 2047), "5da138314db8ce0c98bbf3a0": (13122, 2429),
            "5d2709a003f801723c3251bf": (9345, 2509), "5d27099f03f801723c32511d": (9700, 1855), "5da138364db8ce0c98bc00f1": (5555, 1649), "5da138274db8ce0c98bbd3d2": (6338, 985)}
#site_ids = ["5da1389e4db8ce0c98bd0547"]

len_seq = 5  # number ot time stamps per sequence, with ~2/second, e.g. len5/10/20 = "2.5/5/10sec duration

num_s = 0
for site_id in tqdm(site_ids): # over sites
    num_s += 1
    print(f"Processing #{num_s}: Site-{site_id}")
    record = pickle.load(open(f"./data_out/full24/{site_id}_10k_mix-counts.pkl", "rb"))
    
    main_dfs = []
    gr = record.groupby(["trace"])
    for group in tqdm(gr.groups):  # loop over traces
        trace_dfs = []   #list of sequences/DataFrames

        _group = gr.get_group(group).copy()
        _ = _group.pop("time")
        _ = _group.pop("trace")
        _len = _group.shape[0]
        n_seq = _len//len_seq
        
        # loop over every sequence for a given trace (skip redundant data)
        for i in range(n_seq):
            post_rec = _group[len_seq*i:len_seq*(i+1)].copy().reset_index(drop=True)
            post_rec.loc[0, ["rx", "ry"]] = 0.0
            post_rec["rx_cum"] = post_rec["rx"].cumsum()
            post_rec["ry_cum"] = post_rec["ry"].cumsum()

            trace_dfs.append(post_rec)

        if len(trace_dfs)>0:  # some sequences are available
            main_dfs.append(pd.concat(trace_dfs).reset_index(drop=True)) # cobine list of dataframes into single dataframe

        #break

    main_dfs = pd.concat(main_dfs).reset_index(drop=True) if len(main_dfs)>0 else None
    print(f"Data converted: {round(100*len(main_dfs)/len(record), 1)}%")
    # saving model for a given site_id
    with open(f"./data_out/full24/seq{len_seq}/{site_id}_10k_mix-counts.pkl", "wb") as f:
        pickle.dump(main_dfs, f)
    

  0%|          | 0/24 [00:00<?, ?it/s]

Processing #1: Site-5d27075f03f801723c2e360f


  0%|          | 0/1141 [00:00<?, ?it/s]

Data converted: 97.0%
Processing #2: Site-5dc8cea7659e181adb076a3f


  0%|          | 0/745 [00:00<?, ?it/s]

...

Data converted: 97.3%
Processing #23: Site-5da138364db8ce0c98bc00f1


  0%|          | 0/82 [00:00<?, ?it/s]

Data converted: 97.2%
Processing #24: Site-5da138274db8ce0c98bbd3d2


  0%|          | 0/213 [00:00<?, ?it/s]

Data converted: 92.7%


***
Convert Test Data into RSSI/DELAY DataFrame for Inference Phase
***

In [3]:
train_bssid = pickle.load(open("./data_out/train_24IDs_standardF_bssid_ranks.pkl", "rb"))
test_data = pickle.load(open("./data_out/test_data.pkl", "rb"))

Wifi-based timesteps

In [12]:
parsed_test_data = {}
num_s = 0

for site_id in tqdm(bssid.keys()):  # over sites
    num_s += 1
    #print(f"Processing #{num_s}: Site-{site_id} with {len(test_data[site_id])} traces and {len(bssid[site_id])} bssids")
    
    parsed_test_data[site_id] = {}
    
    col_names = []
    for feature_bssid in bssid[site_id]:  # create features-columns list
        col_names.append(feature_bssid)
        col_names.append(feature_bssid+"_D")
    
    for trace_id in tqdm(test_data[site_id]):  # over traces
        
        wifi_record = test_data[site_id][trace_id].wifi.copy()
        
        accel_record = test_data[site_id][trace_id].acce
        rotate_record = test_data[site_id][trace_id].rotate
        magnetic_record = test_data[site_id][trace_id].magn  
        
        timestamps = wifi_record.time.unique().tolist()
        trace_record_wifi = np.zeros((len(timestamps), len(col_names)))

        for col_id, _ in enumerate(col_names):
            if col_id%2 == 0:
                trace_record_wifi[:, col_id] = int(train_bssid["min_rssi"][site_id])
            else:
                trace_record_wifi[:, col_id] = int(train_bssid["max_delay"][site_id])

        trace_record_wifi = pd.DataFrame(trace_record_wifi, columns=col_names)
        
        trace_record_wifi["time"] = timestamps
        trace_record_wifi["trace"] = trace_id
        timestamps = trace_record_wifi.time.to_numpy()

        # magnetic field intensity
        magnetic_intensity = np.sqrt(np.square(magnetic_record.x_axis.to_numpy()) + np.square(magnetic_record.y_axis.to_numpy()) + np.square(magnetic_record.z_axis.to_numpy()))
        trace_record_wifi["m"] = interp1d(magnetic_record.time.to_numpy(), magnetic_intensity, kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # z_axis rotation angle (i.e. inplane direction)
        trace_record_wifi["r"] = interp1d(rotate_record.time.to_numpy(), rotate_record.z_axis.to_numpy(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # relative x/y steps positions
        step_timestamps, step_indexs, step_acce_max_mins = compute_steps(accel_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy())
        rel_step_positions = pd.DataFrame(compute_rel_positions(compute_stride_length(step_acce_max_mins),
                                                                compute_step_heading(step_timestamps,
                                                                                     compute_headings(rotate_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy()))), 
                                          columns=["time", "x", "y"])
        # recalculate relative steps for a given timestamps(cumsum->interpolate->differences)
        _rt = np.insert(rel_step_positions.time.to_numpy(), 0, 0.0)
        rx_cum = interp1d(_rt, np.insert(rel_step_positions.x.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        ry_cum = interp1d(_rt, np.insert(rel_step_positions.y.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        trace_record_wifi["rx"] = np.diff(rx_cum, prepend=0.0) 
        trace_record_wifi["ry"] = np.diff(ry_cum, prepend=0.0)
        trace_record_wifi["rx_cum"] = rx_cum
        trace_record_wifi["ry_cum"] = ry_cum
        trace_record_wifi.loc[0, ["rx", "ry", "rx_cum", "ry_cum"]] = 0.
                   
        # Take wifi data the nearest to each waypoint
        for j, t0 in enumerate(trace_record_wifi.time):  # over times in trace/for each waypoint

            wifi_min = wifi_record[wifi_record.time == t0]
            wifi_min_bssid_list = wifi_min.bssid.tolist()
            wifi_min_delay_list = [col+"_D" for col in wifi_min_bssid_list]
            
            trace_record_wifi.loc[j, wifi_min_bssid_list] = wifi_min.rssi.to_numpy()
            trace_record_wifi.loc[j, wifi_min_delay_list] = wifi_min.delay.to_numpy().astype(int)

        parsed_test_data[site_id][trace_id] = trace_record_wifi
   
        #break  # only fist trace

    #break  # only first site

  0%|          | 0/24 [00:00<?, ?it/s]

Processing #1: Site-5da1389e4db8ce0c98bd0547 with 13 traces and 1041 bssids
Processing #2: Site-5da138b74db8ce0c98bd4774 with 29 traces and 3546 bssids
Processing #3: Site-5da138764db8ce0c98bcaa46 with 36 traces and 1949 bssids
Processing #4: Site-5dbc1d84c1eb61796cf7c010 with 50 traces and 4544 bssids
Processing #5: Site-5da1383b4db8ce0c98bc11ab with 26 traces and 1554 bssids
Processing #6: Site-5d2709a003f801723c3251bf with 20 traces and 1269 bssids
Processing #7: Site-5a0546857ecc773753327266 with 29 traces and 3433 bssids
Processing #8: Site-5da138274db8ce0c98bbd3d2 with 8 traces and 498 bssids
Processing #9: Site-5d2709b303f801723c327472 with 31 traces and 1934 bssids
Processing #10: Site-5da958dd46f8266d0737457b with 51 traces and 3531 bssids
Processing #11: Site-5d2709bb03f801723c32852c with 34 traces and 2492 bssids
Processing #12: Site-5d2709d403f801723c32bd39 with 51 traces and 2308 bssids
Processing #13: Site-5dc8cea7659e181adb076a3f with 35 traces and 4923 bssids
Processing

In [13]:
with open(f"./data_out/full24/test-10k_mix-counts.pkl", "wb") as f:
    pickle.dump(parsed_test_data, f)

0.55sec-based timesteps

In [4]:
parsed_test_data = {}
num_s = 0
dt = 550 # in milisceonds

for site_id in tqdm(bssid.keys()):  # over sites
    num_s += 1
    #print(f"Processing #{num_s}: Site-{site_id} with {len(test_data[site_id])} traces and {len(bssid[site_id])} bssids")
    
    parsed_test_data[site_id] = {}
    
    col_names = []
    for feature_bssid in bssid[site_id]:  # create features-columns list
        col_names.append(feature_bssid)
        col_names.append(feature_bssid+"_D")
    
    for trace_id in tqdm(test_data[site_id]):  # over traces
        
        wifi_record = test_data[site_id][trace_id].wifi.copy()
        
        accel_record = test_data[site_id][trace_id].acce
        rotate_record = test_data[site_id][trace_id].rotate
        magnetic_record = test_data[site_id][trace_id].magn  
        
        timestamps = wifi_record.time.unique().tolist()
        timestamps = np.linspace(timestamps[0], timestamps[-1], int(1+(timestamps[-1]-timestamps[0])/dt)).tolist()
        
        trace_record_wifi = np.zeros((len(timestamps), len(col_names)))

        for col_id, _ in enumerate(col_names):
            if col_id%2 == 0:
                trace_record_wifi[:, col_id] = int(train_bssid["min_rssi"][site_id])
            else:
                trace_record_wifi[:, col_id] = int(train_bssid["max_delay"][site_id])

        trace_record_wifi = pd.DataFrame(trace_record_wifi, columns=col_names)
        
        trace_record_wifi["time"] = timestamps
        trace_record_wifi["trace"] = trace_id
        timestamps = trace_record_wifi.time.to_numpy()

        # magnetic field intensity
        magnetic_intensity = np.sqrt(np.square(magnetic_record.x_axis.to_numpy()) + np.square(magnetic_record.y_axis.to_numpy()) + np.square(magnetic_record.z_axis.to_numpy()))
        trace_record_wifi["m"] = interp1d(magnetic_record.time.to_numpy(), magnetic_intensity, kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # z_axis rotation angle (i.e. inplane direction)
        trace_record_wifi["r"] = interp1d(rotate_record.time.to_numpy(), rotate_record.z_axis.to_numpy(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        # relative x/y steps positions
        step_timestamps, step_indexs, step_acce_max_mins = compute_steps(accel_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy())
        rel_step_positions = pd.DataFrame(compute_rel_positions(compute_stride_length(step_acce_max_mins),
                                                                compute_step_heading(step_timestamps,
                                                                                     compute_headings(rotate_record[["time", "x_axis", "y_axis", "z_axis"]].to_numpy()))), 
                                          columns=["time", "x", "y"])
        # recalculate relative steps for a given timestamps(cumsum->interpolate->differences)
        
        _rt = np.insert(rel_step_positions.time.to_numpy(), 0, 0.0)
        rx_cum = interp1d(_rt, np.insert(rel_step_positions.x.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        ry_cum = interp1d(_rt, np.insert(rel_step_positions.y.to_numpy(), 0, 0.0).cumsum(), kind="linear", fill_value="extrapolate", copy=False, assume_sorted=True)(timestamps)
        trace_record_wifi["rx"] = np.diff(rx_cum, prepend=0.0) 
        trace_record_wifi["ry"] = np.diff(ry_cum, prepend=0.0)
        trace_record_wifi["rx_cum"] = rx_cum
        trace_record_wifi["ry_cum"] = ry_cum
        trace_record_wifi.loc[0, ["rx", "ry", "rx_cum", "ry_cum"]] = 0.
                   
        # Take wifi data the nearest to each waypoint
        for j, t0 in enumerate(trace_record_wifi.time):  # over times in trace/for each waypoint
            
            deltas = (wifi_record.time - t0).abs()
            tmin = deltas.min()
            
            wifi_min = wifi_record[deltas == tmin]
            wifi_min_bssid_list = wifi_min.bssid.tolist()
            wifi_min_delay_list = [col+"_D" for col in wifi_min_bssid_list]
            
            trace_record_wifi.loc[j, wifi_min_bssid_list] = wifi_min.rssi.to_numpy()
            
# >>>> delays should be adjusted for the time difference  (closest delay may become larger than time_delay_limit for models)
            trace_record_wifi.loc[j, wifi_min_delay_list] = wifi_min.delay.to_numpy().astype(int)
## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        parsed_test_data[site_id][trace_id] = trace_record_wifi
   
        #break  # only fist trace

    #break  # only first site

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
with open(f"./data_out/full24/test-10k_mix-counts_t550.pkl", "wb") as f:
    pickle.dump(parsed_test_data, f)